# 1 этап - загрузка модулей и предобработка текстов дневников

In [2]:
import re
import transliterate

In [72]:
reg = '\[...\]' #Очистка дневников от купюр
reg_links = '[А-ЯЁ][а-яё]*[1-9]{1,5}' #Очистка от сносок
f = open('Diaries.txt')
text = f.read()
f.close()
text = re.sub(reg, '', text)

'''
На заметку: ссылки после фамилий были убраны вручную, в этом куске кода читерство - Ил. Ис. будет заменен на Илис
для облегчения поиска
'''
cheat_text = re.sub('Ил. Ис.', 'Илис', text)

clean_diaries = open(r'./Clean_diaries.txt', 'w')
clean_diaries.write(cheat_text)
clean_diaries.close()

In [73]:
#эта часть кода открывает твой текст, лемматизирует его по определенному принципу (здесь принцип -c - он сохраняет оригинал и при этом рядом пишет лемму)
#и сохраняет его в новом файле, который стоит заранее создать. 
#вначале надо писать путь к программе mystem, потом путь к редактируемому тексту, потом путь к пустому файлу, куда все сохранится
!/Users/tonleon/Downloads/mystem /Users/tonleon/Documents/GitHub/Coursework-2020/Clean_diaries.txt /Users/tonleon/Documents/GitHub/Coursework-2020/text.txt -d -c 

# В этой части создается наш будущий словарь, в котором лежат транслиты в ключах и русские вариации в значениях:

In [74]:
f = open('List_of_russian_surnames.txt', 'r')
list_of_surnames = f.readlines()
f.close
new_list_of_surnames = []
reg = '([А-ЯЁ][а-яё]{2,})'
for line in list_of_surnames:
    line = re.findall(reg, line)
    line += ['\n']
    line = ' '.join(line)
    if line not in new_list_of_surnames:
        new_list_of_surnames.append(line)
        
for surname_index, surname in enumerate(new_list_of_surnames):
    new_list_of_surnames[surname_index] = surname.lower()
    
def processing(surnames):
    surnames = ' '.join(surnames)
    surnames = re.sub('святополк мирский', 'святополк-мирский', surnames)
    surnames = re.sub('мельников печерский', 'мельников-печерский', surnames)
    surnames = re.sub('поляков литовцев', 'поляков-литовцев', surnames)
    surnames = re.sub('кулябко карецкий', 'кулябко-карецкий', surnames)
    surnames = re.sub('овсянико куликовский', 'овсянико-куликовский', surnames)
    surnames = re.sub('соколова кречетова', 'соколова-кречетова', surnames)
    surnames = re.sub('дон аминадо', 'дон-аминадо', surnames)
    surnames = re.sub('бунаков фондаминский', 'бунаков-фондаминский', surnames)
    return surnames

final_list = processing(new_list_of_surnames).split('\n')

In [75]:
def make_dict(final_list, to_str=True):
    eng_2_rus_name = {}
    for name in final_list[1:-1]:

        name_list = name.strip().split(' ')
        eng_name = transliterate.translit(name_list[0], reversed=True) #транслитерируем русскую фамилию
        if to_str:
            eng_2_rus_name[eng_name] = name.strip()
        else:
            eng_2_rus_name[eng_name] = name_list
    return eng_2_rus_name

### * ВОТ ТУТ ПОЯВИЛАСЬ ПЕРЕМЕННАЯ, В КОТОРОЙ ЛЕЖИТ ИСХОДНЫЙ СЛОВАРЬ

In [76]:
dict_surnames = make_dict(final_list, False)

* ОБНАРУЖЕННЫЙ БОНУС: вот так можно генерировать списки и словари

In [79]:
[i for i in range(5) if i%2==0 and i!=0 and i+1==3]

[2]

* собственно, так сгенерирован словарь, в котором в ключах находятся все фамилии и прозвища, которые у нас были, и их аналоги на английском языке

In [80]:
first_rus_2_eng = {rus_surnames[i]: eng_surname \
                   for eng_surname, rus_surnames in dict_surnames.items() \
                   for i in range(len(rus_surnames))}

# Тут начинается код по расставлению тэгов и созданию словаря (под который будет отдана отдельная тетрадка)

In [81]:
f = open(r'text.txt', encoding = "UTF-8") #открываем кусок дневника
text = f.read()
f.close()
dnevnik = text.split('\n') #сплитим кусок по пробелам и видим слово и его лемму
dnevnik_paragraph = []
for string in dnevnik:
    if re.match('^ *?\d', string):
        dnevnik_paragraph.append(string)
    else:
        dnevnik_paragraph.append('<p> '+string+' </p>') #размечаем каждый абзац дневника
        
dnevnik = '\n'.join(dnevnik_paragraph)

dnevnik = dnevnik.split(' ')

* Surname_2_mark - функция, позволяющая расставлять правильно тэги с опорой на наш словарь

In [82]:
def surname_2_mark(surname, original_form):
    return '<persName ref="#'+ first_rus_2_eng[surname] +'" lemma = "'+ surname +'">'+ original_form +'</persName>'

# Вот тут начинается главная обработка с поиском фамилий

In [83]:
reg = "\{\S*\}" #это регулярка для того, что находится в {} скобках

marked_text= []
for string in dnevnik: #работаем с переменной, в которой у нас список со словами и их леммами
    #print ('исходная строка ', string)
    allregs = re.findall(reg, string) #находим в списке подстроки, которые подходят под регулярку
    if allregs: #если нашли такую подстроку, начинаем ее очищать от всякой грязи
        for word in allregs: 
            original_form = re.sub(reg, '', string) #очищаем строки от их лемм и сохраняем в таком виде
            #print ()
            word = word.strip('{}')
            word = word.replace("|", " ")
            word = word.replace("?","")
            
            if ' ' in word and re.match ('[А-ЯЁ]', re.sub('[^А-ЯЁа-яё]','',original_form)): #смотрим на те строки, где у нас подряд написано несколько имен и при этом проверяем, начинаются ли исходные строки, а не леммы, с большой буквы
                list_words = word.split() #превращаем их в список из нескольких элементов
                printed = False
                for n in list_words:
                    n.strip() #каждый элемент списка как бы чистим, превращая таким образом это в строки, которые в двух разных линиях
                    
                    if n in first_rus_2_eng: #ищем подстроки в словаре
                        #print ('case 1')
                        marked_text.append(surname_2_mark(n, original_form))
                        
                        printed = True
                if not printed:
                    #print ('case 2')
                    marked_text.append(original_form)
                   
            elif word in first_rus_2_eng and re.match ('[А-ЯЁ]', re.sub('[^А-ЯЁа-яё]','',original_form)): #возвращаемся к нормальным одиноким словам в строке, ищем их в словаре
                #print ('case 3')
                marked_text.append(surname_2_mark(word, original_form)) #если программа находит, пишет тэг
            
            else:
                #print ('case 4')
                marked_text.append(original_form) #пишем итоговый вариант
    else:
        marked_text.append(string)
    
new_text = ' '.join(marked_text)

#print(marked_text)

In [84]:
first_data = True #создаем глобальную метку того, что вхождение даты первое
start = True #создаем глобальную метку того, что вхождение строки первое

def put_tag(line):
    global first_data, start #передаем глобальные переменные функции
    if start:
        start = False #меняем, потому что он уже прошел первую строчку, и для всех остальных это неправда
        return '<root>' + line
    elif re.match('^ *?\d', line): #есть ошибка, потому что одна из записей начинается с числа, но оно не относится к дате
        if first_data:
            first_data = False #the same case
            return '<note><p><date>'+line+'</date></p>'
        else:
            return '</note><note><p><date>'+line+'</date></p>'
    else:
        return line

In [85]:
def mark_text(text):
    global first_data, start
    first_data = True #создаем глобальную метку того, что вхождение даты первое
    start = True #создаем глобальную метку того, что вхождение строки первое
    clean_list = []
    text_for_dates = text.split('\n')
    for string in text_for_dates:
        clean_list.append(put_tag(string))
    clean_list[-1]+='</note></root>'
   
    return '\n'.join(clean_list)

In [86]:
marked_diary = mark_text(new_text)

In [87]:
final_text = open(r'./Marked_text.xml', 'w')
final_text.write(marked_diary)
final_text.close()

# Попробуем вот здесь разделить текст дневников по годам и сохранить в разные документы

In [88]:
def split_marked_diary_by_years(marked_diary):
    years = re.findall('<date>\d{4}</date>', marked_diary)
    tmp_diary = marked_diary #на всякий случай сделаем копию, с которой будем работать
    diaries_by_years = []
    for year in years:
        diary = tmp_diary.split(year)
        diaries_by_years.append(diary[0])
        tmp_diary = diary[1]
    diaries_by_years.append(tmp_diary)
    return diaries_by_years

In [89]:
lst_of_diaries_by_years = split_marked_diary_by_years(marked_diary)  

## Тут мы каждый дневник из списка lst_of_diaries немного изменяем (т.е. делаем для каждого дневника норомальную разметку с правильными тэгами

In [90]:
for index, diary in enumerate(lst_of_diaries_by_years):
    if index == 0:
        diary_as_lst = diary.split('\n')
        diary_as_lst[-1] = diary_as_lst[-1].replace('<note><p>', '</root>')
        diary_as_str = ('\n').join(diary_as_lst)
        
    elif index == 11:
        diary_as_lst = diary.split('\n')
        diary_as_lst[0] = diary_as_lst[0].replace('</p>', '<root>')
        diary_as_lst[5] = diary_as_lst[5].replace('</note>', '')
        diary_as_str = ('\n').join(diary_as_lst)
    else:
        diary_as_lst = diary.split('\n')
        diary_as_lst[0] = diary_as_lst[0].replace('</p>', '<root>')
        diary_as_lst[-1] = diary_as_lst[-1].replace('<note><p>', '</root>')
        diary_as_str = ('\n').join(diary_as_lst)
        diary_as_str = diary_as_str.replace('</note>', '', 1)
#         re.sub('</note>', '', (re.search('</note>', diary_as_str)).group(0))
    lst_of_diaries_by_years[index] = diary_as_str        

## А здесь мы все сохраняем в новые xml документы

In [91]:
def save_splited_diary_by_years(lst_of_diaries_by_years, start_year=1928, end_year=1940):
    for index, year in enumerate(range(start_year, end_year)):
        file = open('Diary-{}.xml'.format(year), 'w')
        file.write(lst_of_diaries_by_years[index])
        file.close()  

In [92]:
save_splited_diary_by_years(lst_of_diaries_by_years)

* БОНУС: не забудь, как работает функция global

In [93]:
fir = True

def square(x):
    global fir #выходит за пределы функции и находит переменные, которые были заданы ранее
    if fir:
        fir = False
        return x**2 + 1
    else:
        return x**2

In [94]:
square(3)

10

* На память

In [ ]:
list(map(lambda x: 0 if x == 1 else x, [1,1,3,2,5,1]))

# Проблемы, которые осталось решить:
* уникальность ref в словаре транслита (у некоторых людей фамилии одинаковые, типа Степун, и порой для определения, о ком идет речь, в тексте дневника ставятся инициалы, но так работает не во всех случаях)
* ошибки в разметке - две записи с неправильными датами и еще некоторые неразмеченные почему-то имена, типа Левиных (+ после некоторых имен идет цифра - это ссылка на комментарий в издании, это надо почистить)
* надо разделить все дневники на годы, а то пока получается одна мешанина на 10 лет
* надо еще почистить дневники от записей самого редактора, в которых она обозначает, кто именно пишет - Вера или Иван, и периодически пишет про какие-то события, в дневниках не обозначенных.